In [ ]:
from math import log, sqrt
import numpy as np
from scipy.optimize import brentq

def lake_model(b=0.42, q=2.0, mean=0.02, stdev=0.001, alpha=0.4, 
                      delta=0.98, nsamples=100, steps=100, **kwargs):    
    '''runs the lake model for 1 stochastic realisation using specified 
    random seed.
        
    Parameters
    ----------
    b : float
        decay rate for P in lake (0.42 = irreversible)
    q : float
        recycling exponent
    mean : float
            mean of natural inflows
    stdev : float
            standard deviation of natural inflows
    alpha : float
            utility from pollution
    delta : float
            future utility discount rate
    steps : int
            the number of time steps (e.g., days)
    
    '''
    X = np.zeros([steps,], dtype=np.float)
    decisions = np.zeros([steps,], dtype=np.float)
    
    for i in range(steps):
        decisions[i] = kwargs[str(i)]
    
    Pcrit = brentq(lambda x: x**q/(1+x**q) - b*x, 0.01, 1.5)
    
    transformed_mean = log(mean**2 / sqrt(stdev**2 + mean**2))
    transformed_sigma = sqrt(log(1.0 + stdev**2 / mean**2))
    natural_inflows = np.random.lognormal(transformed_mean, transformed_sigma, size=steps)

    for t in range(1, steps):
        X[t] = (1-b)*X[t-1] + X[t-1]**q/(1+X[t-1]**q) + decisions[t-1] + natural_inflows[t-1]

    reliability = np.sum(X < Pcrit)/steps
    utility = np.sum(alpha*decisions*np.power(delta,np.arange(steps)))
    inertia = np.sum(np.abs(np.diff(decisions)) < 0.01)/(steps-1)
    return X, utility, inertia, reliability

In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate

import numpy as np
from scipy.optimize import brentq

cimport cython
cimport numpy as np
from libc.math cimport log, sqrt

ctypedef np.float_t DTYPE_t


@cython.cdivision(True)
@cython.boundscheck(False) 
def cython_lake_model(float b=0.42, float q=2.0, float mean=0.02, 
                      float stdev=0.001, float alpha=0.4, float delta=0.98, 
                      int nsamples=100, int steps=100, **kwargs):    
    '''runs the lake model for 1 stochastic realisation using specified 
    random seed.
        
    Parameters
    ----------
    b : float
        decay rate for P in lake (0.42 = irreversible)
    q : float
        recycling exponent
    mean : float
            mean of natural inflows
    stdev : float
            standard deviation of natural inflows
    alpha : float
            utility from pollution
    delta : float
            future utility discount rate
    steps : int
            the number of time steps (e.g., days)
    seed : int, optional
           seed for the random number generator
    
    '''
    cdef float Pcrit, reliability, utility, inertia, transformed_mean, transformed_sigma
    cdef int t, i
    cdef np.ndarray[DTYPE_t, ndim=1] X = np.zeros([steps,], dtype=np.float)
    cdef np.ndarray[DTYPE_t, ndim=1] decisions = np.zeros([steps,], dtype=np.float)
    cdef np.ndarray[DTYPE_t, ndim=1] natural_inflows 
    
    for i in range(steps):
        decisions[i] = kwargs[str(i)]
    
    Pcrit = brentq(lambda x: x**q/(1+x**q) - b*x, 0.01, 1.5)
    
    transformed_mean = log(mean**2 / sqrt(stdev**2 + mean**2))
    transformed_sigma = sqrt(log(1.0 + stdev**2 / mean**2))
    natural_inflows = np.random.lognormal(transformed_mean, transformed_sigma, size=steps)

    for t in range(1, steps):
        X[t] = (1-b)*X[t-1] + X[t-1]**q/(1+X[t-1]**q) + decisions[t-1] + natural_inflows[t-1]

    reliability = np.sum(X < Pcrit)/steps
    utility = np.sum(alpha*decisions*np.power(delta,np.arange(steps)))
    inertia = np.sum(np.abs(np.diff(decisions)) < 0.01)/(steps-1)
    return X, utility, inertia, reliability

In [ ]:
decisions = {str(i):0 for i in range(100)}

In [ ]:
%%timeit
lake_model(**decisions)

In [ ]:
%%timeit
cython_lake_model(**decisions)

the use of cython here brings a lot less speedup that in the endogenous learning case. The reason for this is that each of the release decisions (100 in total) needs to be a separate keyword argument. This is a requirement imposed by the workbench. To make this easies, I use \*\*kwargs to wrap all decisions into a single dict that I can subsequently use to get the relevant decision. This \*\*kwargs notation is not convertable to C since C does not have variable length arguments. 

# Analysis

In [ ]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant)
from ema_workbench.em_framework.model import Replicator, BaseModel

#instantiate the model
class ReplicatorModel(Replicator, BaseModel):
    pass
lake_model = ReplicatorModel('lakeproblem', function=cython_lake_model)
lake_model.replications = 150

#specify uncertainties
lake_model.uncertainties = [RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('delta', 0.93, 0.99)]

# set levers
lake_model.levers = [RealParameter(str(i), 0, 0.1) for i in range(100)]

def process_p(values):
    values = np.asarray(values)
    values = np.mean(values, axis=0)
    return np.max(values)

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', 
                                     kind=ScalarOutcome.MINIMIZE,
                                     function=process_p
                                    ),
                       ScalarOutcome('utility', 
                                     kind=ScalarOutcome.MAXIMIZE,
                                     function=np.mean
                                    ),
                       ScalarOutcome('inertia', 
                                     kind=ScalarOutcome.MAXIMIZE,
                                     function=np.mean
                                    ),
                       ScalarOutcome('reliability', 
                                     kind=ScalarOutcome.MAXIMIZE,
                                     function=np.mean
                                    )]

# override some of the defaults of the model
lake_model.constants = [Constant('alpha', 0.41),
                        Constant('nsamples', 100),
                        Constant('steps', 100)]

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging, perform_experiments
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(policies=500)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

experiments, outcomes = results

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


x = outcomes['max_P']
y = outcomes['utility']
z = outcomes['reliability']

ax.scatter(x, y, z, depthshade=False)

ax.scatter(x, z, c='lightgrey', zdir='y', zs=2, depthshade=False)
ax.scatter(y, z, c='lightgrey', zdir='x', zs=-0, depthshade=False)
ax.scatter(x, y, c='lightgrey', zdir='z', zs=-0.5, depthshade=False)

ax.set_xlabel("max_P")
ax.set_ylabel("utility")
ax.set_zlabel("reliability")
plt.show()

# reference scenario optimization

In [ ]:
from ema_workbench.em_framework.optimization import to_dataframe

class Callback(object):
    '''Callable object for tracking progress of optimization over generations
    '''

    def __init__(self):
        self.progress = []
        self.archive_history = []

    def __call__(self, optimizer):
        self.progress.append(optimizer.algorithm.archive.improvements)
        
        dvnames = optimizer.problem.parameter_names
        outcome_names = optimizer.problem.outcome_names
        
        self.archive_history.append(to_dataframe(optimizer, dvnames,
                                                 outcome_names))
callback = Callback()

with MultiprocessingEvaluator(lake_model) as evaluator:
    optimization_results = evaluator.optimize(searchover='levers', 
                      nfe=100000, epsilons=[0.05,]*len(lake_model.outcomes),
                      callback=callback, )

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(callback.progress)
ax.set_ylabel('$\epsilon$-progress')
ax.set_xlabel('generations')
plt.show()

In [ ]:
x_opt = optimization_results['max_P']
y_opt = optimization_results['utility']
z_opt = optimization_results['reliability']

In [ ]:
complete_x = np.concatenate((x, x_opt))
complete_y = np.concatenate((y, y_opt))
complete_z = np.concatenate((z, z_opt))
complete_inertia = np.concatenate((outcomes['inertia'], optimization_results['inertia']))
labeling = np.asarray(['random guess',]*500+['optimization',]*z_opt.shape[0])
data = {'max p':complete_x, 'utility':complete_y, 'reliability':complete_z, 
        'inertia':complete_inertia, 'type':labeling}

combined = pd.DataFrame(data)


In [ ]:
sns.pairplot(combined, hue='type')
plt.show()

# animation

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import HTML

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

sns.set_style('white')

x = outcomes['max_P']
y = outcomes['utility']
z = outcomes['reliability']

x_opt = optimization_results['max_P']
y_opt = optimization_results['utility']
z_opt = optimization_results['reliability']

# Axes limits
pholims = (0, 2.6)
utilims = (-0.5, 2.6)
rellims = (0, 1.1)

experiments, outcomes = results

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# setup plot
ax.view_init(elev=20, azim=120)
ax.set_xlim(pholims)
ax.set_ylim(utilims)
ax.set_zlim(rellims)
ax.set_yticks([])
ax.set_xticks([])
ax.set_zticks([])

x_arrow = Arrow3D(pholims, [utilims[1], utilims[1]], [0, 0],
                  mutation_scale = 30, lw=2, arrowstyle="<|-", color="k")
y_arrow = Arrow3D([pholims[0],pholims[0]], utilims, [0, 0],
                  mutation_scale = 30, lw=2, arrowstyle="-|>", color="k")
z_arrow = Arrow3D([0,0], [utilims[1], utilims[1]], rellims,
                  mutation_scale = 30, lw=2, arrowstyle="-|>", color="k")
ax.add_artist(x_arrow)
ax.add_artist(y_arrow)
ax.add_artist(z_arrow)

# initial guesses
x = outcomes['max_P']
y = outcomes['utility']
z = outcomes['reliability']

ax.scatter(x, y, z, c='#3182bd', depthshade=False)

c = '#deebf7' # '#9ecae1'
# c = 'lightgrey'
ax.scatter(x, z, c=c, zdir='y', zs=utilims[1], depthshade=False)
ax.scatter(y, z, c=c, zdir='x', zs=pholims[0], depthshade=False)
ax.scatter(x, y, c=c, zdir='z', zs=-0.1, depthshade=False)

# optimization
archive = callback.archive_history
x_opt = archive[0]['max_P']
y_opt = archive[0]['utility']
z_opt = archive[0]['reliability']

d3_scatter = ax.scatter(x_opt, y_opt, z_opt, c='#e6550d', depthshade=False)

c = '#fee6ce' #'#fdae6b'
xz_shadow = ax.scatter(x_opt, z_opt, c=c, zdir='y', zs=utilims[1], depthshade=False)
yz_shadow = ax.scatter(y_opt, z_opt, c=c, zdir='x', zs=pholims[0], depthshade=False)
xy_shadow = ax.scatter(x_opt, y_opt, c=c, zdir='z', zs=-0.1, depthshade=False)

ax.set_xlabel("max_P", fontsize=20)
ax.set_ylabel("utility", fontsize=20)
ax.set_zlabel("reliability", fontsize=20)

plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

def update(front):
    x_opt = front['max_P'].values
    y_opt = front['utility'].values
    z_opt = front['reliability'].values
    
    d3_scatter._offsets3d = (x_opt, y_opt, z_opt)

    x_offset = np.ones(x_opt.shape)*pholims[0]
    y_offset = np.ones(x_opt.shape)*utilims[1]
    z_offset = np.ones(x_opt.shape)*-0.1

    xz_shadow._offsets3d = (x_opt, y_offset, z_opt)
    yz_shadow._offsets3d = (x_offset, y_opt, z_opt)
    xy_shadow._offsets3d = (x_opt, y_opt, z_offset)
    return d3_scatter

animated = animation.FuncAnimation(fig, update, 
                                   frames=callback.archive_history,
                                   blit=False)
plt.close(animated._fig)
HTML(animated.to_html5_video())

# save everything

In [ ]:
from ema_workbench import save_results
from io import BytesIO, StringIO
import tarfile

base = 'no_learning'
animated.save('./data/{}.mp4'.format(base))
optimization_results.to_csv('./data/{}.cvs'.format(base))
save_results(results, './data/{}_guess.tar.gz'.format(base))
pd.Series(callback.progress).to_csv('./data/{}_progress.csv'.format(base))

file_name = './data/{}_archives.tar.gz'.format(base)
with tarfile.open(file_name, 'w:gz') as z:
    for i, entry in enumerate(callback.archive_history):
        file_buffer = StringIO()
        entry.to_csv(file_buffer)
        
        csv_as_string = file_buffer.getvalue()
        fh = BytesIO(csv_as_string.encode('UTF-8'))
        tarinfo = tarfile.TarInfo(str(i)+'.csv')
        tarinfo.size = len(csv_as_string)
        
        z.addfile(tarinfo, fh)
        